# Parlay User Addresses

This notebook loads parlay markets, filters trades for those markets, and extracts the unique user addresses (maker + taker).

In [ ]:
from pathlib import Path
import polars as pl
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Get contract addresses to exclude
poly_contracts_str = os.getenv("POLY_CONTRACT_ADDRESSES", "")
EXCLUDE_ADDRESSES = set(addr.strip().lower() for addr in poly_contracts_str.split(",") if addr.strip())
print(f"Excluding {len(EXCLUDE_ADDRESSES)} contract addresses: {EXCLUDE_ADDRESSES}")

In [2]:
# Locate parlay markets list
# Note: Running from notebooks/ directory, so parent is project root
project_root = Path.cwd().parent

parlay_candidates = [
    project_root / "processed" / "parlay_markets_api.csv",
    project_root / "processed" / "parlay_markets_manual.csv",
]

parlay_path = next((p for p in parlay_candidates if p.exists()), None)
if parlay_path is None:
    raise FileNotFoundError(
        "No parlay markets CSV found. Run scripts/fetch_parlays_from_api.py first."
    )

print(f"Loading parlay markets from {parlay_path}...")
parlay_df = pl.read_csv(parlay_path, schema_overrides={'token1': pl.Utf8, 'token2': pl.Utf8})
parlay_ids = set(parlay_df.select(pl.col("id")).to_series().to_list())
print(f"✓ Loaded {len(parlay_ids):,} parlay market IDs")
list(parlay_ids)[:10]

Loading parlay markets from /home/junel/random-walk-studio/polymarket/polymarket/processed/parlay_markets_manual.csv...
✓ Loaded 137 parlay market IDs


[247814,
 549898,
 531470,
 549902,
 592915,
 525854,
 539681,
 506408,
 587830,
 518211]

In [3]:
# Locate trades dataset (prefer latest 10M parquet, then full parquet, then CSV)
trade_candidates = [
    project_root / "processed" / "latest_100000000_trades.parquet",
]

trades_path = next((p for p in trade_candidates if p.exists()), None)
if trades_path is None:
    raise FileNotFoundError("No trades dataset found in processed/.")

print(f"Using trades from: {trades_path}")
print(f"File size: {trades_path.stat().st_size / (1024**3):.1f} GB")

Using trades from: /home/junel/random-walk-studio/polymarket/polymarket/processed/latest_100000000_trades.parquet
File size: 4.8 GB


In [4]:
# Filter trades to parlay markets and extract addresses
print("\nFiltering trades for parlay markets...")

if trades_path.suffix == ".parquet":
    trades = pl.scan_parquet(trades_path)
else:
    trades = pl.scan_csv(trades_path)

# Filter to only parlay markets
parlay_trades = trades.filter(pl.col("market_id").is_in(parlay_ids))

# Count parlay trades
num_parlay_trades = parlay_trades.select(pl.len()).collect().item()
print(f"✓ Found {num_parlay_trades:,} parlay trades")

# Extract unique addresses (both makers and takers)
print("\nExtracting unique user addresses...")
addresses = (
    parlay_trades
    .select([pl.col("maker"), pl.col("taker")])
    .unpivot()
    .select(pl.col("value").alias("address"))
    .unique()
    .collect()
)

print(f"✓ Found {len(addresses):,} unique addresses")
addresses.head(10)


Filtering trades for parlay markets...
✓ Found 73,345 parlay trades

Extracting unique user addresses...
✓ Found 11,018 unique addresses


address
str
"""0x2116b148fce58f004cfa449fb7a6…"
"""0x8e3001c4ea2673a69788d7839b27…"
"""0xb95d7aa3263a7b8abed4510c78cf…"
"""0xdff5bd6dff8aba00eb5369bccd8d…"
"""0x24b7fa7688993ce752e517b91edd…"
"""0x842c98ae6e2287b11bae59e6c1e8…"
"""0x67014290ba2a2f8d3f3bf641059b…"
"""0xe9642ed0336ef65b754d6ab08372…"
"""0x2bd5f877604b3d9f9206a83132bb…"


In [5]:
# save parlay trades to CSV
parlay_trades
output_path = project_root / "processed" / "parlay_trades.csv"
print(f"\nSaving parlay trades to {output_path}...")
parlay_trades.collect().write_csv(output_path)
print("✓ Parlay trades saved.")


Saving parlay trades to /home/junel/random-walk-studio/polymarket/polymarket/processed/parlay_trades.csv...
✓ Parlay trades saved.


In [6]:
# Save addresses to CSV
addresses_path = project_root / "processed" / "parlay_user_addresses.csv"
addresses.write_csv(addresses_path)

print(f"\n✅ Saved {len(addresses):,} addresses to:")
print(f"   {addresses_path}")


✅ Saved 11,018 addresses to:
   /home/junel/random-walk-studio/polymarket/polymarket/processed/parlay_user_addresses.csv


In [ ]:
# Aggregate USD volume per address (maker + taker) and sort descending
print("\nAggregating USD volume per address...")
maker_vol = parlay_trades.select(
    pl.col("maker").alias("address"),
    pl.col("usd_amount")
)
taker_vol = parlay_trades.select(
    pl.col("taker").alias("address"),
    pl.col("usd_amount")
)

volume_df = (
    pl.concat([maker_vol, taker_vol])
    .collect()
    .group_by("address")
    .agg(pl.col("usd_amount").sum().alias("total_usd"))
    .sort("total_usd", descending=True)
)

# Filter out contract addresses
if EXCLUDE_ADDRESSES:
    print(f"\nFiltering out {len(EXCLUDE_ADDRESSES)} contract addresses...")
    volume_df = volume_df.filter(~pl.col("address").str.to_lowercase().is_in(EXCLUDE_ADDRESSES))

volume_path = project_root / "processed" / "parlay_user_volume.csv"
volume_df.write_csv(volume_path)

print(f"✅ Aggregated {len(volume_df):,} addresses with volume (after filtering)")
print(f"\nTop 10 by volume:")
print(f"Saved to: {volume_path}")
volume_df.head(10)

## Top 20 Volume Users Analysis

Analyzing the trading behavior of the top 20 users by volume in parlay markets.

**Note:** Contract addresses from POLY_CONTRACT_ADDRESSES in .env are excluded from this analysis.

In [8]:
# Get top 20 users by volume
top_20_users = volume_df.head(20)
print("Top 20 Parlay Users by USD Volume:")
print(top_20_users)
print(f"\nTotal volume from top 20: ${top_20_users.select(pl.col('total_usd').sum()).item():,.2f}")

Top 20 Parlay Users by USD Volume:
shape: (20, 2)
┌─────────────────────────────────┬───────────────┐
│ address                         ┆ total_usd     │
│ ---                             ┆ ---           │
│ str                             ┆ f64           │
╞═════════════════════════════════╪═══════════════╡
│ 0x4bfb41d5b3570defd03c39a9a4d8… ┆ 2.8427e6      │
│ 0xf419573877439e31131f83aba0be… ┆ 246703.58982  │
│ 0xc5d563a36ae78145c45a50134d48… ┆ 231067.420896 │
│ 0x3a8651c42ac19aa3e3141531a298… ┆ 224173.946737 │
│ 0x662ce90c51d613a2975a536272e4… ┆ 208040.569139 │
│ …                               ┆ …             │
│ 0x15691449477aa15e72cdff51c7b6… ┆ 79755.464427  │
│ 0x7c873c11e40ceaaba6385322d49f… ┆ 65516.214298  │
│ 0x5cd5c8d7a17c78d7389d8b87b611… ┆ 63191.050319  │
│ 0x9ef582748baf671ee716075658b8… ┆ 60634.438488  │
│ 0xa9b44dca52ed35e59ac2a6f49d12… ┆ 60316.887929  │
└─────────────────────────────────┴───────────────┘

Total volume from top 20: $5,223,514.45


In [ ]:
# Extract top 20 user addresses (excluding contract addresses)
top_20_addresses = set(top_20_users.select(pl.col("address")).to_series().to_list())
print(f"Top 20 user addresses: {len(top_20_addresses)}")

# Filter parlay trades for top 20 users (either as maker or taker)
print("\nFiltering parlay trades for top 20 users...")
top_20_trades = (
    parlay_trades
    .filter(
        pl.col("maker").is_in(top_20_addresses) | 
        pl.col("taker").is_in(top_20_addresses)
    )
    .collect()
)

print(f"✓ Found {len(top_20_trades):,} trades involving top 20 users")
print(f"\nSample trades:")
top_20_trades.head(10)

In [10]:
# Analyze which markets top 20 users trade in
print("Market breakdown for top 20 users:")
market_stats = (
    top_20_trades
    .group_by("market_id")
    .agg([
        pl.len().alias("num_trades"),
        pl.col("usd_amount").sum().alias("total_volume"),
        pl.col("maker").n_unique().alias("unique_makers"),
        pl.col("taker").n_unique().alias("unique_takers")
    ])
    .sort("total_volume", descending=True)
)

print(f"\n✓ Top 20 users traded in {len(market_stats)} different parlay markets")
print("\nTop 10 markets by volume:")
market_stats.head(10)

Market breakdown for top 20 users:

✓ Top 20 users traded in 84 different parlay markets

Top 10 markets by volume:


market_id,num_trades,total_volume,unique_makers,unique_takers
i64,u32,f64,u32,u32
664644,7956,911621.632442,2550,1119
581796,3669,502784.176541,1138,571
564607,3048,465618.529202,1236,214
592915,2916,353815.795572,734,461
587216,2848,315785.555753,718,260
609512,2399,315352.112504,579,393
620390,1673,183460.931882,467,129
560348,1436,156793.921533,545,122
620400,1967,134290.327696,514,329


In [11]:
# Join with parlay markets to get market details
market_details = (
    market_stats
    .join(parlay_df, left_on="market_id", right_on="id", how="left")
    .select([
        "market_id",
        "question",
        "num_trades",
        "total_volume",
        "unique_makers",
        "unique_takers"
    ])
    .sort("total_volume", descending=True)
)

print("Top 10 Parlay Markets by Volume (Top 20 Users):")
market_details.head(10)

Top 10 Parlay Markets by Volume (Top 20 Users):


market_id,question,num_trades,total_volume,unique_makers,unique_takers
i64,str,u32,f64,u32,u32
664644,"""Nothing Ever Happens: November""",7956,911621.632442,2550,1119
581796,"""Nothing Ever Happens: Septembe…",3669,502784.176541,1138,571
564607,"""Astronomer Divorce Parlay""",3048,465618.529202,1236,214
592915,"""Nothing Ever Happens: Military…",2916,353815.795572,734,461
587216,"""Will the Fed cut-cut-cut in 20…",2848,315785.555753,718,260
609512,"""Nothing Ever Happens: Russia E…",2399,315352.112504,579,393
620390,"""Zohran Mamdani Parlay""",1673,183460.931882,467,129
560348,"""Nothing Ever Happens: July""",1436,156793.921533,545,122
620400,"""Nothing Ever Happens: World Le…",1967,134290.327696,514,329


In [12]:
# Detailed trade information for top 20 users
print("Trade Details Summary:")
trade_summary = top_20_trades.select([
    pl.col("timestamp"),
    pl.col("market_id"),
    pl.col("maker"),
    pl.col("taker"),
    pl.col("price"),
    pl.col("token_amount"),
    pl.col("usd_amount"),
    pl.col("maker_direction"),
    pl.col("taker_direction")
]).sort("usd_amount", descending=True)

print(f"\nTop 20 largest trades by USD amount:")
trade_summary.head(20)

Trade Details Summary:

Top 20 largest trades by USD amount:


timestamp,market_id,maker,taker,price,token_amount,usd_amount,maker_direction,taker_direction
str,i64,str,str,f64,f64,f64,str,str
"""2025-08-22T02:30:59.000000""",578474,"""0x8f053ac26c46b27f304cb51ae35d…","""0x4bfb41d5b3570defd03c39a9a4d8…",0.979941,34159.77,33474.5746,"""BUY""","""SELL"""
"""2025-08-22T02:46:11.000000""",578474,"""0x9c704d41ef1ae81a2c8ad91cfee0…","""0x4bfb41d5b3570defd03c39a9a4d8…",0.98,24887.04,24389.2992,"""BUY""","""SELL"""
"""2025-08-22T02:46:11.000000""",578474,"""0x8f053ac26c46b27f304cb51ae35d…","""0x9c704d41ef1ae81a2c8ad91cfee0…",0.98,24887.04,24389.2992,"""SELL""","""BUY"""
"""2025-11-26T12:23:03.000000""",664644,"""0x8f0da308939ca4441713ce87d708…","""0x4bfb41d5b3570defd03c39a9a4d8…",0.939962,22341.3183,20999.999985,"""BUY""","""SELL"""
"""2025-09-26T19:51:51.000000""",581796,"""0xf0d8d8c22d1e80257deebd45706c…","""0x24c8cf69a0e0a17eee21f69d2975…",0.96,19902.99,19106.8704,"""BUY""","""SELL"""
…,…,…,…,…,…,…,…,…
"""2025-09-28T16:51:19.000000""",581796,"""0x4d73a91d2434a6041011283931b7…","""0x24c8cf69a0e0a17eee21f69d2975…",0.96,12043.11,11561.3856,"""SELL""","""BUY"""
"""2025-08-05T20:28:47.000000""",564607,"""0x5bffeb3a5fa46f29689efeabad05…","""0x4bfb41d5b3570defd03c39a9a4d8…",0.949136,11932.49686,11325.559997,"""BUY""","""SELL"""
"""2025-09-29T18:37:31.000000""",581796,"""0x24c8cf69a0e0a17eee21f69d2975…","""0x4bfb41d5b3570defd03c39a9a4d8…",0.968984,11205.61,10858.05409,"""BUY""","""SELL"""


In [13]:
# Per-user trading statistics
print("Individual User Trading Statistics:")

# Create a combined view where we identify if top_20 user was maker or taker
maker_trades = top_20_trades.filter(pl.col("maker").is_in(top_20_addresses)).select([
    pl.col("maker").alias("user"),
    pl.col("market_id"),
    pl.col("usd_amount"),
    pl.lit("maker").alias("role")
])

taker_trades = top_20_trades.filter(pl.col("taker").is_in(top_20_addresses)).select([
    pl.col("taker").alias("user"),
    pl.col("market_id"),
    pl.col("usd_amount"),
    pl.lit("taker").alias("role")
])

user_trades = pl.concat([maker_trades, taker_trades])

user_stats = (
    user_trades
    .group_by("user")
    .agg([
        pl.len().alias("num_trades"),
        pl.col("market_id").n_unique().alias("unique_markets"),
        pl.col("usd_amount").sum().alias("total_volume"),
        pl.col("usd_amount").mean().alias("avg_trade_size"),
        pl.col("usd_amount").max().alias("max_trade_size")
    ])
    .sort("total_volume", descending=True)
)

print("\nDetailed statistics for top 20 users:")
user_stats

Individual User Trading Statistics:

Detailed statistics for top 20 users:


user,num_trades,unique_markets,total_volume,avg_trade_size,max_trade_size
str,u32,u32,f64,f64,f64
"""0x4bfb41d5b3570defd03c39a9a4d8…",26328,83,2.8427e6,107.970862,33474.5746
"""0xf419573877439e31131f83aba0be…",383,1,246703.58982,644.134699,17686.3162
"""0xc5d563a36ae78145c45a50134d48…",2187,1,231067.420896,105.654971,8961.9952
"""0x3a8651c42ac19aa3e3141531a298…",3497,15,224173.946737,64.104646,2460.0
"""0x662ce90c51d613a2975a536272e4…",1040,15,208040.569139,200.039009,6999.999999
…,…,…,…,…,…
"""0x15691449477aa15e72cdff51c7b6…",156,4,79755.464427,511.252977,6412.0113
"""0x7c873c11e40ceaaba6385322d49f…",93,1,65516.214298,704.475423,9966.83112
"""0x5cd5c8d7a17c78d7389d8b87b611…",219,8,63191.050319,288.543609,3988.0


In [14]:
# Per-user market breakdown - which markets does each top 20 user trade in?
print("Market breakdown per user:")

user_market_breakdown = (
    user_trades
    .group_by(["user", "market_id"])
    .agg([
        pl.len().alias("num_trades"),
        pl.col("usd_amount").sum().alias("market_volume")
    ])
    .join(parlay_df.select(["id", "question"]), left_on="market_id", right_on="id", how="left")
    .select([
        "user",
        "market_id",
        "question",
        "num_trades",
        "market_volume"
    ])
    .sort(["user", "market_volume"], descending=[False, True])
)

print(f"\n✓ Created market breakdown for {user_market_breakdown.select(pl.col('user').n_unique()).item()} users")
print(f"\nShowing all markets for each top 20 user:")
user_market_breakdown

Market breakdown per user:

✓ Created market breakdown for 20 users

Showing all markets for each top 20 user:


user,market_id,question,num_trades,market_volume
str,i64,str,u32,f64
"""0x06e8cb40376ff9f06d926e71a074…",581796,"""Nothing Ever Happens: Septembe…",88,32435.961054
"""0x06e8cb40376ff9f06d926e71a074…",664644,"""Nothing Ever Happens: November""",212,29313.505361
"""0x06e8cb40376ff9f06d926e71a074…",618439,"""Nothing Ever Happens: October""",47,21933.39101
"""0x06e8cb40376ff9f06d926e71a074…",609512,"""Nothing Ever Happens: Russia E…",79,18514.503683
"""0x06e8cb40376ff9f06d926e71a074…",578474,"""UFC Fight Night: Mingyang and …",6,11319.472
…,…,…,…,…
"""0xfcf2378f20cf408d077c21e73127…",587216,"""Will the Fed cut-cut-cut in 20…",121,10660.910573
"""0xfcf2378f20cf408d077c21e73127…",612820,"""Nothing Ever Happens: Airdrops…",155,6319.926279
"""0xfcf2378f20cf408d077c21e73127…",618439,"""Nothing Ever Happens: October""",60,5422.399162


In [20]:
# Helper function to view markets for a specific user
def show_user_markets(user_address, top_n=10):
    """Show top N markets for a specific user"""
    user_data = (
        user_market_breakdown
        .filter(pl.col("user") == user_address)
        .sort("market_volume", descending=True)
        .head(top_n)
    )
    
    total_volume = user_data.select(pl.col("market_volume").sum()).item()
    print(f"\nUser: {user_address}")
    print(f"Total volume: ${total_volume:,.2f}")
    print(f"Top {min(top_n, len(user_data))} markets:")
    return user_data

# Show top 5 markets for the #1 user
top_user = top_20_users.select(pl.col("address")).head(1).item()
show_user_markets("0x3a8651c42ac19aa3e3141531a298abc72f51dea8", top_n=15)


User: 0x3a8651c42ac19aa3e3141531a298abc72f51dea8
Total volume: $224,173.95
Top 15 markets:


user,market_id,question,num_trades,market_volume
str,i64,str,u32,f64
"""0x3a8651c42ac19aa3e3141531a298…",592915,"""Nothing Ever Happens: Military…",1148,79723.670373
"""0x3a8651c42ac19aa3e3141531a298…",609512,"""Nothing Ever Happens: Russia E…",642,51624.575023
"""0x3a8651c42ac19aa3e3141531a298…",620400,"""Nothing Ever Happens: World Le…",673,39849.375631
"""0x3a8651c42ac19aa3e3141531a298…",664644,"""Nothing Ever Happens: November""",297,23103.63179
"""0x3a8651c42ac19aa3e3141531a298…",581796,"""Nothing Ever Happens: Septembe…",534,17675.501854
…,…,…,…,…
"""0x3a8651c42ac19aa3e3141531a298…",604508,"""Nothing Ever Happens: AI Editi…",11,422.468308
"""0x3a8651c42ac19aa3e3141531a298…",620390,"""Zohran Mamdani Parlay""",12,331.037121
"""0x3a8651c42ac19aa3e3141531a298…",679582,"""Trump's Christmas Wishlist Par…",4,171.950441


In [16]:
# Save per-user market breakdown
user_market_breakdown_path = project_root / "processed" / "top_20_user_market_breakdown.csv"
user_market_breakdown.write_csv(user_market_breakdown_path)

print(f"✅ Saved per-user market breakdown to:")
print(f"   {user_market_breakdown_path}")
print(f"\nThis file shows which parlay markets each of the top 20 users traded in,")
print(f"with the number of trades and volume per user per market.")

✅ Saved per-user market breakdown to:
   /home/junel/random-walk-studio/polymarket/polymarket/processed/top_20_user_market_breakdown.csv

This file shows which parlay markets each of the top 20 users traded in,
with the number of trades and volume per user per market.


In [17]:
# Save top 20 user trades and statistics
top_20_trades_path = project_root / "processed" / "top_20_parlay_user_trades.csv"
user_stats_path = project_root / "processed" / "top_20_user_statistics.csv"
market_details_path = project_root / "processed" / "top_20_user_markets.csv"

top_20_trades.write_csv(top_20_trades_path)
user_stats.write_csv(user_stats_path)
market_details.write_csv(market_details_path)

print(f"✅ Saved top 20 user analysis:")
print(f"   - Trades: {top_20_trades_path}")
print(f"   - User Statistics: {user_stats_path}")
print(f"   - Market Details: {market_details_path}")

✅ Saved top 20 user analysis:
   - Trades: /home/junel/random-walk-studio/polymarket/polymarket/processed/top_20_parlay_user_trades.csv
   - User Statistics: /home/junel/random-walk-studio/polymarket/polymarket/processed/top_20_user_statistics.csv
   - Market Details: /home/junel/random-walk-studio/polymarket/polymarket/processed/top_20_user_markets.csv
